#### SBERT
- BERT 모델: 문장 이해용 Encoder
    - 문장 쌍 비교
- SBERT 모델: 문장 의미 임베딩
    - 벡터의 비교용

In [2]:
# 라이브러리 설치
# !pip install sentence-transformers

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util

In [4]:
# 모델을 로드 -> 두 개의 문장을 비교 (코사인 유사도를 통해 두 문장이 얼마나 근접한지 확인)
# 다목적 한국어 SBERT
model_name = 'jhgan/ko-sroberta-multitask'
# 또는
# 문장 유사도 특화
model_name2 = 'BM-K/KoSimCSE-roberta-multitask'

sbert = SentenceTransformer(model_name)
sbert2 = SentenceTransformer(model_name2)

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\student\.cache\huggingface\hub\models--jhgan--ko-sroberta-multitask. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. F

In [5]:
# 최대 토큰의 길이를 설정
sbert.max_seq_length = 256
sbert2.max_seq_length = 256

In [6]:
# 두 개의 문장을 비교
doc1 = "이 카메라는 색감이 자연스럽고 배터리도 오래간다"
doc2 = "배터리 성능이 좋고 사진 품질이 뛰어나다"

In [7]:
# 두 개의 문장을 임베딩하고 코사인 유사도 계산
# sbert인 경우
with torch.inference_mode():
    emb1 = sbert.encode(doc1, convert_to_tensor=True, normalize_embeddings=True)
    emb2 = sbert.encode(doc2, convert_to_tensor=True, normalize_embeddings=True)
# 코사인 유사도 계산
cos_sim = util.cos_sim(emb1, emb2).item()
print('유사도: ', round(cos_sim, 4))
# 유사도 0.6948

유사도:  0.6948


In [8]:
# 두 개의 문장을 임베딩하고 코사인 유사도 계산
# sbert2인 경우
with torch.inference_mode():
    emb1 = sbert2.encode(doc1, convert_to_tensor=True, normalize_embeddings=True)
    emb2 = sbert2.encode(doc2, convert_to_tensor=True, normalize_embeddings=True)
# 코사인 유사도 계산
cos_sim = util.cos_sim(emb1, emb2).item()
print('유사도: ', round(cos_sim, 4))
# 유사도 0.734

유사도:  0.734


In [9]:
emb1.shape

torch.Size([768])

In [10]:
sentences = [
    '삼성전자 주가가 올랐다',
    '코스피가 상승 마감했다',
    '비가 많이 와서 항공편이 취소됐다'
]
# 0, 1번째 문장과 비슷한 다른 문장을 sentences와 비교해보기
with torch.inference_mode():
    embs = sbert2.encode(sentences, convert_to_tensor= True, normalize_embeddings= True)
    # 임베드 벡터 3개, 컬럼 768개

# 세 문장에서의 유사도 확인
sim_metrix = util.cos_sim(embs, embs)
print(sim_metrix)

tensor([[ 1.0000,  0.3585, -0.0052],
        [ 0.3585,  1.0000,  0.1199],
        [-0.0052,  0.1199,  1.0000]])


In [11]:
new_sentence = "증시가 강세였다"
# 임베딩
new_emb = sbert2.encode(new_sentence, convert_to_tensor= True, normalize_embeddings= True)

# 유사도가 높은 상위 n개 확인
top_n = 2
hits = torch.topk(
    util.cos_sim(new_emb, embs).squeeze(0),     # squeeze로 차원 하나 떨어뜨림
    k= top_n
)
hits

torch.return_types.topk(
values=tensor([0.6161, 0.6014]),
indices=tensor([0, 1]))

In [13]:
for score, idx in zip( hits.values.tolist(), hits.indices.tolist() ):
    print(f"{sentences[idx]} | score: {score:.3f}")

삼성전자 주가가 올랐다 | score: 0.616
코스피가 상승 마감했다 | score: 0.601
